In [1]:
# %run models.py

In [2]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [3]:
device = torch.device("cuda")
kwargs = {'num_workers': 1, 'pin_memory': True} 

# hyper params
batch_size = 64
latent_size = 20
epochs = 10

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./mnist', train=True, download=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./mnist', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False, **kwargs)

In [5]:
def one_hot(labels, class_size):
    targets = torch.zeros(labels.size(0), class_size)
    for i, label in enumerate(labels):
        targets[i, label] = 1
    return targets.to(device)

In [19]:
class CVAE(nn.Module):
    def __init__(self, feature_size, latent_size, class_size):
        super(CVAE, self).__init__()
        self.feature_size = feature_size
        self.class_size = class_size

        # encode
        self.fc1  = nn.Linear(feature_size + class_size, 400)
        self.fc21 = nn.Linear(400, latent_size)
        self.fc22 = nn.Linear(400, latent_size)

        # decode
        self.fc3 = nn.Linear(latent_size + class_size, 400)
        self.fc4 = nn.Linear(400, feature_size)

        self.elu = nn.ELU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x, c): # Q(z|x, c)
        '''
        x: (bs, feature_size)
        c: (bs, class_size)
        '''
        inputs = torch.cat([x, c], 1) # (bs, feature_size+class_size)
        h1 = self.elu(self.fc1(inputs))
        z_mu = self.fc21(h1)
        z_var = self.fc22(h1)
        return z_mu, z_var

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z, c): # P(x|z, c)
        '''
        z: (bs, latent_size)
        c: (bs, class_size)
        '''
        inputs = torch.cat([z, c], 1) # (bs, latent_size+class_size)
        h3 = self.elu(self.fc3(inputs))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x, c):
        #mu, logvar = self.encode(x.view(-1, 28*28), c)  #for MNIST
        mu, logvar = self.encode(x.view(-1, 128*128), c)  #for SANE
        z = self.reparameterize(mu, logvar)
        return self.decode(z, c), mu, logvar


In [20]:
# create a CVAE model
#model = CVAE(28*28, latent_size, 10).to(device)   #for MNIST
model = CVAE(128*128, latent_size, 6).to(device)  #for SANE

optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    #BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')  #for MNIST
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 16384), reduction='sum')  #for SANE
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD



In [21]:
from torchsummary import summary

summary(model, [(128*128,), (6,)])  #summary(model, [(feature_size,), (class_size,)])


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 400]       6,556,400
               ELU-2                  [-1, 400]               0
            Linear-3                   [-1, 20]           8,020
            Linear-4                   [-1, 20]           8,020
            Linear-5                  [-1, 400]          10,800
               ELU-6                  [-1, 400]               0
            Linear-7                [-1, 16384]       6,569,984
           Sigmoid-8                [-1, 16384]               0
Total params: 13,153,224
Trainable params: 13,153,224
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forward/backward pass size (MB): 0.26
Params size (MB): 50.18
Estimated Total Size (MB): 50.81
----------------------------------------------------------------


In [9]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        labels = one_hot(labels, 10)
        recon_batch, mu, logvar = model(data, labels)
        optimizer.zero_grad()
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.detach().cpu().numpy()
        optimizer.step()
        #if batch_idx % 20 == 0:
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                #epoch, batch_idx * len(data), len(train_loader.dataset),
                #100. * batch_idx / len(train_loader),
                #loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [10]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, labels) in enumerate(test_loader):
            data, labels = data.to(device), labels.to(device)
            labels = one_hot(labels, 10)
            recon_batch, mu, logvar = model(data, labels)
            test_loss += loss_function(recon_batch, data, mu, logvar).detach().cpu().numpy()
            if i == 0:
                n = min(data.size(0), 5)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(-1, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


In [11]:
from tqdm.auto import tqdm

for epoch in tqdm(range(1, epochs + 1)):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            c = torch.eye(10, 10).cuda()
            sample = torch.randn(10, 20).to(device)
            sample = model.decode(sample, c).cpu()
            save_image(sample.view(10, 1, 28, 28),
                       'sample_' + str(epoch) + '.png')

  0%|          | 0/10 [00:00<?, ?it/s]

====> Epoch: 1 Average loss: 144.5022
====> Test set loss: 119.6992
====> Epoch: 2 Average loss: 117.1439
====> Test set loss: 113.1256
====> Epoch: 3 Average loss: 111.9242
====> Test set loss: 109.1891
====> Epoch: 4 Average loss: 108.8352
====> Test set loss: 107.3600
====> Epoch: 5 Average loss: 106.8328
====> Test set loss: 106.0862
====> Epoch: 6 Average loss: 105.4024
====> Test set loss: 104.6744
====> Epoch: 7 Average loss: 104.4542
====> Test set loss: 103.3462
====> Epoch: 8 Average loss: 103.6405
====> Test set loss: 102.7561
====> Epoch: 9 Average loss: 103.0769
====> Test set loss: 102.7040
====> Epoch: 10 Average loss: 102.6399
====> Test set loss: 101.9089
